---
<div align="justify">
    <h1><strong>An automation approach of the mechanical design and manufacturing of ball mills</strong></h1>
    <h2>Juan D. Argüello$^a$, Omar A. Gélvez$^b$, Jairo R. Martínez$^c$, Diego C. Durán$^c$, Elena E. Stashenko$^c$</h2>
    <foot><i>$^a$Universidad Industrial de Santander, Escuela de Ingeniería Mecánica, Centro Nacional de Investigaciones para la Agroindustrialización de Especies Vegetales Aromáticas y Medicinales Tropicales - CENIVAM. Bucaramanga, Colombia.</i></foot>
    <br>
    <foot><i>$^b$Universidad Industrial de Santander, Escuela de Ingeniería Mecánica, Grupo de Investigación en Energía y Medio Ambiente - GIEMA. Bucaramanga, Colombia.</i></foot>
    <br>
    <foot><i>$^c$Universidad Industrial de Santander, Escuela de Química, Centro de Investigación en Biomoléculas - CIBIMOL y Centro Nacional de Investigaciones para la Agroindustrialización de Especies Vegetales Aromáticas y Medicinales Tropicales - CENIVAM. Bucaramanga, Colombia.</i></foot>
</div>

---

<div align="center">
    <h2><strong>Abstract</strong></h2>
    Engineering design is taking a new step forward with technological advances. <i>Open source</i> tools, such as Python, make it possible to develop calculation and validation algorithms to automate design procedures. Ball mills stands out as a solution for crushing processes, specially in the civil industry, which its design procedure requires the study of the cynematic of the balls and the dynamical interaction between them and the mill. Numerical simulations are usually used as a validation tool, but also as an optimization one. For this particular case, there were used two numerical methods: the <i>Discrete Element Method</i> (DEM), used to validate the cynematic theoretical study, optimize it and to predict the impact distribution energy over the mill; and the <i>Finite Element Method</i> (FEM), which was used to predict, and guarantee, an adecuate structural behaviour of the mill. The present work also create automatic engineering reports (both TeX and PDF formats), CAD drawings of the designed ball mill (dxf format) and an automatic quantification of the required materials for manufacturing. All of the above can be summarized in three principal stages, or chapters: the first one is related to the theoretical study, the second one to the numerical simulations and the last one to the obtained results.
</div>

_Keywords:_ ball mill, automation, mechanical design, FEM, DEM, FEniCS, Python.

---

## __1. Introduction__

### _1.1. Background and Motivation_

<div align="justify">
    In recent years, there have been reported several studies where automation has been used in engineering design. For the production of nuclear energy, it can be found different studies, such as: the design of reactor cores$^{[1]}$, from where Hyun Kim <i>et al</i> uses an artificial neural network to provide, and evaluate, new types of geometries; Nomen <i>et al</i> developed a methodology to create the design of IFMIF LIPAc beam dump shielding$^{[2]}$, which provides an acelerator-based, D-Li neutron source to produce energy of high intensity by radiation; and Kim J <i>et al</i> created an autonomous operation system$^{[3]}$ with artificial intelligence which perform the control functions needed for the emergency operation of a nuclear power plant.
    <br>
    <br>
    In civil engineering, there is a specialized journal known as "Automation in Construction", which has published different studies related to automatic structural and seismic designs. In mechanical design, it can be found studies related with automaton in the automotive industry, like:  
</div>

## __1. Proceso de extracción__

<div align="justify">
    La planta de extracción a diseñar pertenece a una cadena de producción destinada a la comercialización de <i>aceite esencial</i>, <i>extracto</i> y <i>material de compostaje</i>, como se muestra en la Figura 1.
</div>

<div align="center">
    <img src="Images/bio.PNG" style="width: 500px;" />
</div>
<div align="center">
    <i>Figura 1.</i> Esquema General de la biofábrica.
</div>

<div align="justify">
    La planta a diseñar presenta tres etapas:
</div>

* _Preprocesamiento:_ consiste en la disminución del tamaño de partícula del material vegetal.
* _Elución y Filtrado:_ etapa en donde el material procesado entra en contacto con un solvente, que posee la capacidad de atraer el extracto, y se realiza el filtrado de la mezcla entre el _material procesado_ y la _mezcla solvente - extracto._ 
* _Separación de sustancias:_ proceso en donde se recupera el solvente procedente la mezcla solvente - extracto. 

<div align="justify">
    Un esquema detallado del proceso de extracción se puede apreciar en la Figura 2.
</div>

<div align="center">
    <img src="Images/Productividad/Esquema.PNG" style="width: 800px;" />
</div>
<div align="center">
    <i>Figura 2.</i> Esquema productivo de la planta de extracción.
</div>

## __2. Especificaciones y Requerimientos__

<div align ="justify">
    El sistema de preprocesamiento consiste de un <i>Sistema de trituración</i> y de un <i>Sistema de transmisión de potencia</i>.
</div>

### 2.1. Requerimientos

<div align="justify">
    Con base a las pruebas experimentales desarrolladas y a las especifaciones técnicas dadas, el subsistema debe cumplir con las siguientes condiciones:
</div>

* Capacidad de procesamiento de $20 \left[kg/lote \right]$.
* Relación másica entre el material vegetal y el agente dispersante es de 1 a 8, respectivamente.
* La transmisión debe ser por correas y trenes de engranajes.
* Material vegetal con 16% de humedad.
* Tamaño de partícula medio de entrada: $850 \left[ \mu m \right]$.
* Tamaño final objetivo: $250 \left[ \mu m\right]$.

### 2.2. Especificaciones

<div align="justify">
     El molino de bolas a diseñar se rige bajo los siguientes criterios:
</div>

* Relación longitud - diámetro de 1.5 a 3.
* Agente dispersante: arena de río $\left(\rho _{AD} = 1500 \left[kg/m^3 \right] \right)$.
* El material de las bolas debe ser de aluminio, por tratarse de material orgánico $\left(\rho _{Al} = 2700 \left[kg/m^3 \right] \right)$.
* El volumen ocupado por la carga del molino (material vegetal y las bolas) esté entre el 40% y el 60% del volumen total del tambor.
* El volumen ocupado por las bolas debe estar entre el 10% y el 30% del volumen total del molino.
* La velocidad rotacional de operación debe ser el 80% de la velocidad de rotación crítica.
* La densidad de las hojas secas de _Lippia origanoides_ es de $100 \left[kg / m^3 \right]$. 
* Factores de seguridad entre 2 y 4.

## __3. Diseño Funcional__

<div align="justify">
    El diseño funcional del sistema busca definir:
</div>

* Potencia de funcionamiento.
* Diámetro del tambor.
* Longitud del tambor.
* Velocidad de rotación.
* Número de bolas.

### 3.1. Datos

<div align="justify">
    Los datos utilizados para el <strong>diseño</strong> son:
</div>

In [1]:
from App.Data import ReqEs
data = ReqEs.Datos()
data

NOTA: el cambio de estos valores permitirá plantear un nuevo diseño escalable, principalmente al modificar la <u><i>capacidad</i></u> de la planta.

### 3.2. Volumen del tambor $\rightarrow$ Diámetro y Longitud

<div align="justify">
    El volumen del tambor (ver Figura 3) es directamente proporcional a la suma de los volúmenes ocupados por la arena de río (agente dispersante), el material vegetal y las bolas, e inversamente proporcional al porcentaja de volumen ocupado por <i>todo</i> el material. 
    <div align="center">
        <img src="Images/Preprocesamiento/volumen.PNG" style="width: 500px;" />
    </div>
    <div align="center">
        <i>Figura 3.</i> Relación masa - volumen.
    </div>
</div>

<div align="justify">
    Para garantizar una trituración eficiente y efectiva (mayor energía de impacto), se sugiere el diseño con un volumen ocupado por la carga de hasta el $40\%$.
    $$
    \begin{equation}
        V = \frac{V_{AD} + V_{mv} + V_{Tb}}{P}
        \tag{1}
        \label{VT}
    \end{equation}
    $$
    De la Ecuación \ref{VT}, $V$ es el volumen total del tambor, $V_{AD}$ es el volumen ocupado por el agente dispersante, $V_{mv}$ es el volumen ocupado por el material vegetal, $V_{Tb}$ correponde al volumen del total de las bolas y $P$ al porcentaje del volumen total ocupado por la carga.
</div>

In [2]:
from IPython.display import HTML, display
import pandas as pd
from App.Data.Read import Read
from App.Preprocesamiento.Funcional.Dimensiones import *
datos = Read(data)
res = {'Vol':{}}
res['Vol'], vol = Volumenes(Read(data))()

volumenes = pd.DataFrame(vol)
display(HTML("El <strong>volumen total</strong> del tambor es de $" + 
             str(vol['Valor $[L]$'][3]) + " [L]$"))
volumenes

,Tipo,Valor $[L]$
0,$V_{AD}$,106.67
1,$V_{MV}$,200.00
2,$V_{Tb}$,102.22
3,$V_{T}$,1022.22


<div align="justify">
    Conociendo el volumen del tambor, se puede proceder a calcular el diámetro y longitud del mismo de la siguiente manera:
    $$
    \begin{equation}
        V = \frac{\pi}{4} D^2 L
        \tag{2}
        \label{volCil}
    \end{equation}
    $$
</div>

In [3]:
res['D'], res['L'] = dim(res['Vol']['V'], datos['Esp']['Relación Longitud - Diámetro'])
display(HTML("El diámetro $D$ es de: $" + str(round(res['D'],2)) + 
             " [m]$ y la longitud del tambor $L$ es de: $" + str(round(res['L'],2)) + " [m]$"))

### 3.3. Velocidad de rotación del molino

<div align="justify">
    El análisis para determinar la velocidad de rotación óptima se basa en principio de que la velocidad de rotación mínima deba hacer que las bolas se mantengan adheridas a la superficie del molino. 
</div>

<div align="center">
        <img src="Images/Preprocesamiento/DBola.PNG" style="width: 400px;" />
</div>
<div align="center">
    <i>Figura 4.</i> Dinámica de una bola.
</div>

<div align="justify">
    Realizando sumatoria de fuerzas en la dirección radial, se obtiene:
    $$
    \begin{equation}
        \sum F_r = 0 = C - W cos \, \alpha \\
        \rightarrow \boxed{C = W cos \, \alpha}
        \tag{3}
        \label{Fr}
    \end{equation}
    $$
    La dinámica de una bola está definida por:
    $$
    \begin{equation}
        \vec{C} = m \vec{a_p} 
        \tag{4}
        \label{dinB}
    \end{equation}
    $$
    Dónde: $m$ corresponde a la masa de la bola y $\vec{a_p}$ al vector aceleración. La aceleración absoluta es equivalente a la suma de tres aceleraciones: aceleración rotacional, aceleración relativa y la aceleración de Coriolis.
</div>

<div align="justify">
    $$
    \begin{equation}
        \vec{a_p} = \dot{\vec{V_p}} = \dot{\vec{\omega}} \times \vec{r} + \vec{\omega} \times \left(\vec{\omega} \times \vec{r} \right) + 2 \vec{\omega} \times \left(\dot{\vec{r}} \right) _{rotacional} + \left(\dot{\vec{r}} \right) _{fijo}
        \tag{5}
        \label{apG}
    \end{equation}
    $$
    La velocidad de rotación del molino es constante, lo que implica que la aceleración de la partícula se comporte bajo la siguiente relación matemática:
    $$
    \begin{equation}
        \vec{a_p} = \vec{\omega} \times \left(\vec{\omega} \times \vec{r} \right)
        \tag{6}
        \label{ap}
    \end{equation}
    $$
    Relacionando las Ecuaciones \ref{Fr}, \ref{dinB} y \ref{ap}, se tiene:
    $$
    \begin{equation}
        C = W cos \, \alpha = m \left(\omega ^2 r \right) = \left(\frac{W}{g} \right) \left[ \left(\frac{V}{r} \right) ^2 r \right]  \rightarrow \\
        W cos \, \alpha = \frac{W \left(2\pi r N(\alpha) \right)^2}{rg} \rightarrow \boxed{N(\alpha) = \sqrt{\frac{g cos \, \alpha}{4 \pi ^2 r}}}
        \tag{7}
        \label{Nalpha}
    \end{equation}
    $$
    En la Ecuación \ref{Nalpha}, se puede detallar que el valor máximo al que debe girar el molino de bolas se encuentra cuando $\alpha = 0$.
    $$
    \begin{equation}
        N_{max} = \sqrt{\frac{g}{4 \pi ^2 r}}
        \tag{8}
        \label{N}
    \end{equation}
    $$
</div>

In [4]:
import math
general = {'g':{'valor':9.81, 'units':'m/s2'}}
N_max = (general['g']['valor']/(4*math.pi**2*(res['D']/2)))**(1/2)
res['N'] = N_max*(datos['Esp']\
                  ['Relación de velocidad operacional respecto a la crítica [%]']/100)*60
display(HTML("La velocidad de rotación crítica $N_{máx}$ es de $" + str(round(N_max,3)) +
            " [rev/s] \\rightarrow" + str(round(N_max*60,3)) + " [rpm]$. "
             "Aplicando el criterio de velocidad operativa ($ " +
             str(datos['Esp']['Relación de velocidad operacional respecto a la crítica [%]']) +
             " \% $ de la velocidad crítica)" +
            " se tiene que la velocidad de rotación del tambor $N$ debe ser de $" + 
             str(round(res['N'],2)) +
            " [rpm]$."))

### 3.5. Número de bolas

<div align="justify">
    Stashenko <i>et al.</i>$^{[1]}$ estimaron que bolas de $2.5$, $4$ y $7 [cm]$ de diámetro presentaban una distribución de energía de impacto adecuada para el proceso de trituración requerido por el sistema de preprocesamiento. Para ello, se subdivide el volumen total de bolas $V_{Tb}$ en cada uno de los tres tipos de bola. El número de bolas por tipo se estima de la siguiente manera:
    $$
    \begin{equation}
        N_b = \frac{V_{Tb}}{\frac{4}{3}\pi \left(\frac{D_b}{2} \right) ^2}
        \tag{9}
        \label{Nb}
    \end{equation}
    $$
</div>

In [5]:
from math import floor
from IPython.display import Markdown
res['NBolas'] = {}
V = (res['Vol']['V_Tb']/3,)
display(HTML("Número de bolas:"))
for tipo in [2.5, 4, 7]:
    Vb = (4/3)*math.pi*(tipo/(2*100))**2
    res['NBolas'][str(tipo)] = {}
    res['NBolas'][str(tipo)]['Cant'] = floor(V[0]/Vb)
    res['NBolas'][str(tipo)]['masa'] = res['NBolas'][str(tipo)]['Cant']*\
        datos['Esp']['Densidad de las bolas [kg/m3]']*Vb
    display(Markdown("* $" + str(tipo) + "[cm]$: " + str(res['NBolas'][str(tipo)]['Cant'])))

* $2.5[cm]$: 52

* $4[cm]$: 20

* $7[cm]$: 6

### 3.6. Potencia teórica

<div align="justify">
    Para el cálculo de la potencia mínima que garantice el funcionamiento del sistema de preprocesamiento, se desarrolla el análisis dinámico mostrado en la Figura 5.
</div>

<div align="center">
    <img src="Images/Preprocesamiento/CM.PNG" style="width: 500px;" />
</div>
<div align="center">
    <i>Figura 5.</i> Diagrama de cuerpo libre del molino.
</div>

<div align="justify">
    Conociendo la velocidad de rotación, se puede calcular la potencia como:
    $$
    \begin{equation}
        Pot = T \omega \\
        T = F.S \, W\, d\, sin\left(\alpha \right) 
        \tag{10}
        \label{pot}
    \end{equation}
    $$
    Dónde: $F.S$ es el factor de seguridad, $W$ es la carga del molino y $d$ es la distancia al centro de masa.
</div>

#### 3.6.1. Distancia al centro de masa

<div align="justify">
    La distancia al centro de masa, se evalúa de la siguiente forma:
    $$
    \begin{equation}
        d = \sqrt{\overline{x}^2 + \overline{y}^2} \\
        \overline{x} A = \int x \, dA \\
        \overline{y} A = \int y \, dA
        \tag{11}
        \label{dcm}
    \end{equation}
    $$
    El área de una región circular se calcula como:
    $$
    \begin{equation}
        A = \frac{1}{2} \theta r^2
        \tag{12}
    \end{equation}
    $$
    El ángulo $\alpha$ que permite tener la mayor energía de impacto es de $45°$. Para convertir la Ecuación \ref{dcm} en coordenadas polares:
    $$
    \begin{equation}
        x = r cos \, \theta \\
        y = r sin \, \theta \\
        dA = r \, dr \, d\theta
    \end{equation}
    $$
</div>

In [6]:
from sympy import *
#Datos...
r, theta = symbols('r, \\theta', real=True, positive=True)
A, xbar, ybar,d = symbols("A, \\bar{x}, \\bar{y},d")
dA = r
x = r*cos(theta)
y = r*sin(theta)

#Solución
AA = integrate(dA, (r, 0, r), (theta,-pi/2,pi/2))
xxbar = integrate(x*dA, (r, -r,0), (theta,(5/4)*pi,pi/4))/AA
yybar = integrate(y*dA, (r, -r,0), (theta,(5/4)*pi,pi/4))/AA
dd = sqrt(xxbar**2 + yybar**2)

#Ecuaciones
Area = Eq(A,Integral(dA, (r, -r, 0), (theta,(5/4)*pi,pi/4)))
display(Markdown("1. Área:"))
display(Area)
display(Eq(A, AA))

1. Área:

Eq(A, Integral(r, (r, -r, 0), (\theta, 1.25*pi, pi/4)))

Eq(A, pi*r**2/2)

In [7]:
xx = Eq(xbar, (1/A)*Integral(x*dA, (r, -r,0), (theta,(5/4)*pi,pi/4)))
display(Markdown("2. $\\bar{x}$:"))
display(xx)
display(Eq(xbar, xxbar))

2. $\bar{x}$:

Eq(\bar{x}, Integral(r**2*cos(\theta), (r, -r, 0), (\theta, 1.25*pi, pi/4))/A)

Eq(\bar{x}, 2*sqrt(2)*r/(3*pi))

In [8]:
yy = Eq(ybar, (1/A)*Integral(y*dA, (r, -r,0), (theta,(5/4)*pi,pi/4)))
display(Markdown("3. $\\bar{y}$:"))
display(yy)
display(Eq(ybar, yybar))
#Solución
yybar = integrate(y*dA, (r, -r,0), (theta,(5/4)*pi,pi/4))/AA
dd = sqrt(xxbar**2 + yybar**2)
display(Markdown("4. distancia:"))
display(Eq(d, sqrt(xbar**2 + ybar**2)))
display(Eq(d, dd))

3. $\bar{y}$:

Eq(\bar{y}, Integral(r**2*sin(\theta), (r, -r, 0), (\theta, 1.25*pi, pi/4))/A)

Eq(\bar{y}, -2*sqrt(2)*r/(3*pi))

4. distancia:

Eq(d, sqrt(\bar{x}**2 + \bar{y}**2))

Eq(d, 4*r/(3*pi))

#### 3.6.2. Cálculo de la potencia

In [9]:
d = dd.subs({r:res['D']/2}).evalf()
W = datos['Req']['Capacidad [kg/bache]']*(1+datos['Req']['Relación másica AD - MV'])
for key in res['NBolas']:
    W += res['NBolas'][key]['masa']
W = W*9.81
T = datos['Esp']['Factor de seguridad']*W*d*math.sin(math.radians(45))
w = res['N']*(2*math.pi/60)
res['Pot'] = T*w*(1.34102/1000)
display(HTML("La <strong>potencia teórica</strong> es de $ " + str(round(res['Pot'],2)) + " [hp]$"))

In [ ]:
W

### 3.7. Resumen de resultados

<div align="justify">
    Los resultados obtenidos se pueden apreciar a continuación:
</div>

In [10]:
from App.Preprocesamiento.Funcional.Resultados import *
result = Ress(res)
pd.DataFrame(result)

,Resultados
Diámetro [m],0.954
Longitud [m],1.431
Potencia [hp],6.087
Velocidad de rotación [rpm],34.648
Volumen [m3],1.022
